In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime
from scipy.sparse import diags

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [2]:
from helpers import load_csv_data

DATA_TRAIN_PATH = 'data/train.csv'

y, X1, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)

In [3]:
def standardize(x, std_x = None, mean_x = None, ignore_first = True):
    """Standardize the original data set."""
    x = np.copy(x)
    if type(mean_x) == type(None):
        mean_x = np.mean(x, axis=0)
    x = x - mean_x
    if ignore_first:
        x[:,0] = 1
    if type(std_x) == type(None):
        std_x = np.std(x, axis=0)
    for i in range(std_x.shape[0]):
        if std_x[i] > 0: x[:, i] = x[:, i] / std_x[i]
    return x, mean_x, std_x

In [4]:
X, _, _ = standardize(X1, ignore_first = False)
print(y.shape, X.shape)

(5000,) (5000, 30)


## Prepare cost and prediction functions

In [38]:
len(np.array([1,2,3]).shape)

1

In [45]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    
    assert(len(y.shape) == 1)
    assert(len((X @ w).shape) == 1)
    assert(y.shape == (X @ w).shape)
    
    return np.sum(np.maximum(0, 1 - np.multiply(y, X @ w))) + lambda_ / 2 * (w.T @ w)

In [29]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    
    return np.mean((X @ w > 0) == (y > 0))

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [10]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    
    return (-num_examples * y_n * x_n * np.sign(np.maximum(0, 1 - y_n * x_n.T @ w)) + lambda_ * w)

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [11]:
X.shape

(5000, 30)

In [14]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 100000
    gamma = 1e-1
    lambda_ = 0.1
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}, acc={a}".format(i=it, c=cost, a=calculate_accuracy(y, X, w)))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=9106565.82805988, acc=0.6232
iteration=10000, cost=644652.0054721062, acc=0.6444
iteration=20000, cost=530722.7162662051, acc=0.6476
iteration=30000, cost=475563.27252475184, acc=0.648
iteration=40000, cost=439614.3167058405, acc=0.6506
iteration=50000, cost=414964.7366741004, acc=0.6486
iteration=60000, cost=396205.30404099496, acc=0.6474
iteration=70000, cost=380439.4896503361, acc=0.651
iteration=80000, cost=367838.9408537437, acc=0.65
iteration=90000, cost=356772.23181420425, acc=0.6534
training accuracy = 0.6558


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [92]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n, gamma):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
    # w/o tricks
    #Y = diags(y)
    #alpha[n] = old_alpha_n + 1 - 1 / lambda_ * (Y @ X @ X.T @ Y @ alpha)[n]
    #w = 1 / lambda_ * X.T @ Y @ alpha
    
    # full optimization
    alpha_n_delta = lambda_ * (1. - y_n * x_n.T @ w) / (x_n.T @ x_n)
    
    #alpha_n_delta = (1 - y_n * x_n.dot(w)) * gamma
    
    #(Y @ X @ X.T @ Y)[n] @ alpha = lambda_
    
    if old_alpha_n + alpha_n_delta > 1:
        alpha[n] = 1
        alpha_n_delta = 1 - old_alpha_n
    elif old_alpha_n + alpha_n_delta < 0:
        alpha[n] = 0
        alpha_n_delta = -old_alpha_n
    else: alpha[n] = old_alpha_n + alpha_n_delta
    
    #print(alpha_n_delta)
    
    w += 1. / lambda_ * x_n * y_n * alpha_n_delta
    
    return w, alpha

In [93]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    #Y = diags(y)
    #return np.sum(alpha) - 0.5 / lambda_ * alpha.T @ Y @ X @ X.T @ Y @ alpha#return np.sum(alpha) - 0.5 / lambda_ * alpha.T @ Y @ X @ X.T @ Y @ alpha
    return np.sum(alpha) - 0.5 * lambda_ * w.T @ w

In [94]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 1000000
    lambda_ = 1
    gamma = 1e-10

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n, gamma)
        #assert(np.max(alpha) <= 1)
        #assert(np.min(alpha) >= 0)
        #assert(np.allclose(w, 1. / lambda_ * X.T @ diags(y) @ alpha))
        #assert(len(alpha.shape) == 1)
        #assert(len(w.shape) == 1)
            
        if it % 100000 == 0:
            accuracy = calculate_accuracy(y, X, w)
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f alpha_comp:%d alpha_norm:%.5f w_norm:%.5f acc:%.5f'%(
                    it, primal_value, dual_value, duality_gap, np.sum(alpha > 0), np.linalg.norm(alpha),
            np.linalg.norm(w), accuracy))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:4965.85228, dual:0.03037, gap:4965.82191 alpha_comp:1 alpha_norm:0.06073 w_norm:0.24644 acc:0.50120
iteration=100000, primal:3536.37529, dual:2517.96792, gap:1018.40737 alpha_comp:4579 alpha_norm:44.06510 w_norm:2.35840 acc:0.71540
iteration=200000, primal:3272.24829, dual:3006.21853, gap:266.02976 alpha_comp:4112 alpha_norm:51.93818 w_norm:2.95712 acc:0.72440
iteration=300000, primal:3231.13710, dual:3096.73872, gap:134.39838 alpha_comp:3843 alpha_norm:53.80751 w_norm:3.07849 acc:0.73780
iteration=400000, primal:3177.86788, dual:3125.45677, gap:52.41111 alpha_comp:3659 alpha_norm:54.57878 w_norm:3.13827 acc:0.73080
iteration=500000, primal:3185.43596, dual:3137.55701, gap:47.87894 alpha_comp:3560 alpha_norm:54.98855 w_norm:3.08897 acc:0.73740
iteration=600000, primal:3163.58360, dual:3144.24595, gap:19.33765 alpha_comp:3499 alpha_norm:55.24676 w_norm:3.04529 acc:0.73700
iteration=700000, primal:3166.60601, dual:3148.60874, gap:17.99727 alpha_comp:3447 alpha_norm:55